In [1]:
!pip install SimpleITK nibabel pydicom gdown

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Example: Accessing a file in your Google Drive
SunnyBrook_dataset_path = '/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook'
print(f"File path: {SunnyBrook_dataset_path}")

# To get the current working directory:
import os
current_directory = os.getcwd()
print(f"\nCurrent working directory: {current_directory}")

# To list files in a directory:
import os
directory_to_list = '/content/drive/My Drive'  # Replace with your desired directory
try:
  files = os.listdir(SunnyBrook_dataset_path)
  print(f"\nFiles and directories in {SunnyBrook_dataset_path}:")
  for file in files:
    print(file)
except FileNotFoundError:
  print(f"Error: Directory not found at {SunnyBrook_dataset_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File path: /content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook

Current working directory: /content

Files and directories in /content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook:
scd_patientdata.csv
scd_capmodels.zip
scd_manualcontours.zip
SCD_IMAGES_01.zip
SCD_IMAGES_05.zip
SCD_IMAGES_04.zip
SCD_IMAGES_03.zip
SCD_IMAGES_02.zip
SCD0000601
Generated_Masks
SCD0000501
SCD0000101
SCD0000801
SCD0000401
SCD0000701
SCD0000301
SCD0000201
SCD0001301
SCD0001801
SCD0000901
SCD0001001
SCD0001101
SCD0001501
SCD0001201
SCD0001401
SCD0001601
SCD0001701
SCD0002401
SCD0002701
SCD0002301
SCD0002001
SCD0002501
SCD0002201
SCD0002101
SCD0002601
SCD0002801
SCD0001901
SCD0003101
SCD0002901
SCD0003801
SCD0003401
SCD0003701
SCD0003001
SCD0003301
SCD0003201
SCD0003601
SCD0003501
SCD0004201
SCD_CAPModels
SCD0004001
SCD0004401
SCD_ManualContours
SCD0003901
SCD00045

In [4]:
import os
import re
import sys
import logging
import importlib.util
import numpy as np
import nibabel as nib
import SimpleITK as sitk
import scipy.interpolate as spi
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
from tqdm import tqdm
from typing import Tuple, Optional, List
import shutil


# Configuration
class Config:
    # BASE_PATH = Path(os.path.abspath(os.path.join(os.getcwd(), "../../Data/SunnyBrook"))) # Locally
    BASE_PATH = Path(SunnyBrook_dataset_path) # Google Drive
    # ORGANIZED_PATH = Path(os.path.abspath(os.path.join(os.getcwd(), "../../Data/SunnyBrook/sunnybrook_Organized")))  # Output directory
    # PROCESSED_PATH = Path(os.path.abspath(os.path.join(os.getcwd(), "../../Data/SunnyBrook/sunnybrook_processed")))  # Output directory
    ORGANIZED_PATH = Path(os.path.join(SunnyBrook_dataset_path, "sunnybrook_Organized"))  # Output directory
    PROCESSED_PATH = Path(os.path.join(SunnyBrook_dataset_path, "sunnybrook_processed"))
    TARGET_SPACING = (1.0, 1.0)  # (x, y)
    TARGET_SHAPE = (512, 512)     # (height, width)
    MAX_PATIENTS = None              # Set to None for all patients
    SEED = 42                     # For reproducible sampling
    START_PATIENT_ID = 151  # Start patient numbering from 151

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

In [5]:
def organize_sunnybrook_dataset(raw_dir: Path, processed_dir: Path) -> None:
    """
    Organize Sunnybrook dataset into a standardized structure.
    - Copies DICOM files directly into patient folders (no CINESAX subfolder)
    - Filters folders to only include those matching SCD + 7 digits
    """
    processed_dir.mkdir(exist_ok=True)

    # Regex pattern to match "SCD" followed by exactly 7 digits
    scd_pattern = re.compile(r'^SCD\d{7}$')
    patient_counter = Config.START_PATIENT_ID

    for scd_folder in sorted(raw_dir.glob("SCD*")):
        # Skip non-matching folders (e.g., SCD_CAPModels, SCD000350)
        if not scd_pattern.match(scd_folder.name):
            logger.debug(f"Skipping invalid folder: {scd_folder.name}")
            continue

        # Create patient folder (e.g., patient151)
        patient_id = f"patient{patient_counter:03d}"
        patient_dir = processed_dir / patient_id
        patient_dir.mkdir(exist_ok=True)

        # Copy ALL CINESAX DICOM files directly into patient folder
        cinesax_files = list(scd_folder.glob("CINESAX*/*.dcm"))
        print(cinesax_files)
        for file in cinesax_files:
            try:
                shutil.copy2(file, patient_dir / file.name)
                logger.debug(f"Copied {file.name} to {patient_dir}")
            except Exception as e:
                logger.error(f"Failed to copy {file.name}: {str(e)}")

        logger.info(f"Organized {scd_folder.name} -> {patient_id}")
        patient_counter += 1
        # Create visualizations folder
        visualizations_dir = processed_dir / "visualizations"
        visualizations_dir.mkdir(exist_ok=True)

        # Save some random DICOM images as PNG
        np.random.seed(Config.SEED)
        sample_files = np.random.choice(cinesax_files, min(5, len(cinesax_files)), replace=False)

        for file in sample_files:
            try:
                img = sitk.ReadImage(str(file))
                array = sitk.GetArrayFromImage(img)[0]  # Get the 2D slice
                plt.imshow(array, cmap='gray')
                plt.axis('off')
                output_file = visualizations_dir / f"{file.stem}.png"
                plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
                plt.close()
                logger.info(f"Saved visualization: {output_file}")
            except Exception as e:
                logger.error(f"Failed to save visualization for {file.name}: {str(e)}")

In [6]:
def resample_to_target(slice_sitk: sitk.Image, target_spacing: Tuple[float, float]) -> sitk.Image:
    """
    Resample slice using scipy.ndimage.zoom.
    Returns a SimpleITK image with correct metadata.
    """
    # Get original metadata
    original_spacing = slice_sitk.GetSpacing()
    original_origin = slice_sitk.GetOrigin()

    # Calculate resize factor (X, Y only)
    resize_factor = np.array([
        original_spacing[0] / target_spacing[0],
        original_spacing[1] / target_spacing[1]
    ])

    # Convert to numpy array (preserve axis order)
    image_np = sitk.GetArrayFromImage(slice_sitk)  # Shape: (Z, Y, X) -> but Z=1 for 2D

    # Remove singleton dimension if needed
    if image_np.shape[0] == 1:
        image_np = image_np[0]  # Shape: (Y, X)

    # Clip to valid range
    image_np = np.clip(image_np, np.percentile(image_np, 1), np.percentile(image_np, 99))

    # Apply interpolation
    resampled_np = zoom(image_np, resize_factor[::-1], order=1)  # Reverse for (Y, X) axes

    # Check for NaN/Inf values
    if np.isnan(resampled_np).any() or np.isinf(resampled_np).any():
        logger.error("NaN/Inf detected after resampling!")
        resampled_np = np.nan_to_num(resampled_np)

    # Create new SimpleITK image
    resampled_sitk = sitk.GetImageFromArray(resampled_np)

    # Set metadata correctly
    resampled_sitk.SetSpacing((target_spacing[0], target_spacing[1]))
    resampled_sitk.SetOrigin(original_origin)

    return resampled_sitk

def normalize_slice(slice_np: np.ndarray) -> np.ndarray:
    """
    Normalize a slice using robust min-max normalization (0-1 range).
    Handles edge cases where the image is constant or invalid.
    """
    p1, p99 = np.percentile(slice_np, [1, 99])

    # Log input image statistics
    logger.debug(f"Input image min/max: {np.min(slice_np)}, {np.max(slice_np)}")
    logger.debug(f"Percentiles: p1={p1}, p99={p99}")

    # Handle edge case: constant image or invalid percentiles
    if p99 - p1 == 0:
        logger.warning(f"Constant image detected: p1={p1}, p99={p99}. Returning zero-filled array.")
        return np.zeros_like(slice_np, dtype=np.float32)

    # Normalize and clip to [0, 1]
    normalized = np.clip((slice_np - p1) / (p99 - p1), 0, 1).astype(np.float32)

    # Check for NaN/Inf values
    if np.isnan(normalized).any() or np.isinf(normalized).any():
        logger.error("NaN/Inf values detected after normalization!")
        normalized = np.nan_to_num(normalized)

    return normalized

def pad_to_shape(slice_np: np.ndarray, target_shape: Tuple[int, int]) -> np.ndarray:
    """Symmetrically pad slice to target shape."""
    pads = [(max(0, (ts - s) // 2), max(0, ts - s - (ts - s) // 2))
            for s, ts in zip(slice_np.shape, target_shape)]
    return np.pad(slice_np, pads, mode='constant')

def monotonic_zoom_interpolate(image_np, resize_factor):
    """
    Apply zoom interpolation using scipy.ndimage.zoom.
    """
    return zoom(image_np, resize_factor, order=1)  # order=1 for linear interpolation

In [7]:
def preprocess_sunnybrook_cinesax(processed_dir: Path, output_dir: Path) -> None:
    """
    Preprocess CINESAX sequences and save slices.

    Args:
        processed_dir: Directory containing organized patient folders with DICOM files.
        output_dir: Directory to save the processed .npy files.
    """
    output_dir.mkdir(exist_ok=True)  # Create output directory if it doesn't exist
    patient_count = 0
    max_patients = Config.MAX_PATIENTS

    for patient_dir in sorted(processed_dir.glob("patient*")):
        if max_patients is not None and patient_count >= max_patients:
            logger.info(f"Reached max patient limit ({max_patients})")
            break

        # Get all DICOM files in patient folder
        dicom_files = sorted(patient_dir.glob("*.dcm"))
        if not dicom_files:
            logger.warning(f"No DICOM files found in {patient_dir.name}")
            continue

        # Create output folder for this patient
        patient_output_dir = output_dir / patient_dir.name
        patient_output_dir.mkdir(exist_ok=True)

        # Group into slices (20 files = 1 slice with 20 time frames)
        slices = [dicom_files[i:i+20] for i in range(0, len(dicom_files), 20)]
        # logger.info(f"Found {len(slices)} slices in {patient_dir.name}")

        for z, slice_files in enumerate(slices):
            # Load time frames for this slice
            time_frames = []
            for f in slice_files:
                try:
                    img = sitk.ReadImage(str(f))
                    array = sitk.GetArrayFromImage(img)
                    time_frames.append(array)
                except Exception as e:
                    logger.error(f"Failed to load {f.name}: {str(e)}")
                    continue

            if not time_frames:
                logger.error(f"No valid DICOM files in slice {z} for {patient_dir.name}")
                continue

            for t, frame in enumerate(time_frames):
                try:
                    # Resample -> Normalize -> Pad
                    frame_sitk = sitk.GetImageFromArray(frame)

                    # Resample to target spacing
                    resampled = resample_to_target(frame_sitk, Config.TARGET_SPACING)
                    resampled_np = sitk.GetArrayFromImage(resampled)

                    normalized = normalize_slice(resampled_np)

                    padded = pad_to_shape(normalized, Config.TARGET_SHAPE)

                    # Save as numpy file in the output directory
                    output_file = patient_output_dir / f"{patient_dir.name}_t{t:02d}_z{z:02d}.npy"
                    np.save(output_file, padded.astype(np.float32))
                    # logger.info(f"Saved {output_file.name} in {patient_output_dir}")
                except Exception as e:
                    logger.error(f"Failed to process/save {patient_dir.name}_t{t:02d}_z{z:02d}: {str(e)}")
                    continue

        patient_count += 1
        logger.info(f"Processed {patient_dir.name} [{patient_count}/{max_patients}]")

In [8]:
def validate_processing(output_dir: Path, num_samples: int = 5) -> None:
    """Validate processed data with multiple checks."""
    # Collect all .npy files from patient folders
    npy_files = list(output_dir.glob("patient*/*.npy"))
    assert len(npy_files) > 0, "No processed files found!"

    # Check array properties of the first file
    sample = np.load(npy_files[0])
    assert sample.shape == Config.TARGET_SHAPE, f"Shape mismatch: {sample.shape}"
    assert sample.dtype == np.float32, f"Dtype mismatch: {sample.dtype}"

    # Value range check (0-1 for min-max normalization)
    min_val, max_val = sample.min(), sample.max()
    if not (0 <= min_val <= max_val <= 1):
        logger.warning(f"Value range error: [{min_val}, {max_val}]")
        logger.warning("This may indicate a normalization issue. Proceeding with validation.")

    # Check for NaN/Inf values
    assert not np.isnan(sample).any(), "NaN values detected in the sample"
    assert not np.isinf(sample).any(), "Inf values detected in the sample"

    logger.info("Validation passed!")
    visualize_samples(output_dir, num_samples)

def visualize_samples(output_dir: Path, num_samples: int = 5) -> None:
    """Visualize random samples with before/after comparison (DICOM)."""
    visualization_dir = output_dir / "visualizations"
    visualization_dir.mkdir(exist_ok=True)
    # logger.info(f"Saving visualizations to: {visualization_dir}")

    npy_files = list(output_dir.glob("patient*/*.npy"))
    if not npy_files:
        logger.warning("No .npy files found for visualization.")
        return

    np.random.seed(Config.SEED)
    sample_files = np.random.choice(npy_files, min(num_samples, len(npy_files)), replace=False)

    for sf in sample_files:
        parts = sf.stem.split('_')
        patient_id = parts[0]
        time_frame = int(parts[1][1:])
        slice_index = int(parts[2][1:])

        # Locate original DICOM folder
        logger.info(f"Processing {sf}")
        original_folder = find_original_image(sf)
        if not original_folder:
            logger.warning(f"Original DICOM folder not found for {sf.name}")
            continue

        # Load DICOM series for the slice
        try:
            # Get all DICOM files for this slice (20 files per slice)
            dicom_files = sorted(original_folder.glob("*.dcm"))[slice_index*20 : (slice_index+1)*20]
            time_frames = [sitk.GetArrayFromImage(sitk.ReadImage(str(f))) for f in dicom_files]
            orig_slice = time_frames[time_frame][0]  # Get the specific time frame and remove extra dimension
        except Exception as e:
            logger.error(f"Failed to load DICOM for {sf.name}: {str(e)}")
            continue

        # Load processed numpy file
        processed = np.load(sf)

        # Plot comparison
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        ax1.imshow(orig_slice, cmap='gray')
        ax1.set_title(f"Original DICOM\nSlice {slice_index}, Frame {time_frame}")
        ax2.imshow(processed, cmap='gray')
        ax2.set_title(f"Processed\n{sf.name}")

        # Save visualization
        output_file = visualization_dir / f"{sf.stem}_comparison.png"
        fig.savefig(output_file, bbox_inches="tight", dpi=300)
        plt.close(fig)
        # logger.info(f"Saved visualization: {output_file}")

def find_original_image(processed_path: Path) -> Optional[Path]:
    """Locate original DICOM folder from processed numpy path."""
    relative_path = processed_path.relative_to(Config.PROCESSED_PATH)
    original_path = Config.ORGANIZED_PATH / relative_path.parent
    if original_path.exists():
        logger.info(f"Found original path for {processed_path} -> {original_path}")
        return original_path
    else:
        logger.warning(f"Original path not found for {processed_path}")
        return None



In [9]:
# Organize the dataset
organize_sunnybrook_dataset(Config.BASE_PATH, Config.ORGANIZED_PATH)

[PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0173.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0004.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0067.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0009.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0054.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0102.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0080.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/SunnyBrook/SCD0000101/CINESAX_300/IM-0003-0031.dcm'), PosixPath('/content/drive/My Drive/GP_Data_Folder/GP_Data-Sets/

In [10]:
# Preprocess CINESAX sequences
preprocess_sunnybrook_cinesax(Config.ORGANIZED_PATH, Config.PROCESSED_PATH)

In [11]:
validate_processing(Config.PROCESSED_PATH, num_samples=100)